In [86]:
from top_github_scraper import (get_top_repo_urls, get_top_repos, get_top_contributors, 
get_top_user_urls, get_top_users)
#import datapane as dp 
import pandas as pd 
from tqdm import tqdm 
from folium import plugins
import geopandas
from geopy.geocoders import Nominatim
import folium
from folium.plugins import Search
import requests
import os
USERNAME = os.getenv("GITHUB_USERNAME")
TOKEN = os.getenv("GITHUB_TOKEN")

In [85]:
keywords = ["machine learning", 'api']

In [99]:
def get_repo_info(keywords, stop=10):
    repos = []
    for k in keywords:
        print(k)
        new_repos = get_top_repo_urls(k, stop_page=stop)
        print(len(new_repos))
        repos = repos + new_repos
        print(len(repos))
    
    all_repo_info = dict()
    info_to_scrape = ['name',"stargazers_count", "forks_count", 'subscribers_count', 'topics', 'language', 'created_at','updated_at']
    for repo in repos:
        repo_url = repo
        repo_info_url = f"https://api.github.com/repos{repo_url}"
        repo_info = requests.get(repo_info_url, auth=(USERNAME, TOKEN)).json()
        repo_name = repo_info['id']
        repo_important_info = {}
        for info in info_to_scrape:
            repo_important_info[info] = repo_info[info]
        all_repo_info[repo_name] = repo_important_info
    repo_df = pd.DataFrame.from_dict(all_repo_info, orient='index', columns=info_to_scrape)
    return repo_df

def topic_relationship_table(repo_df):
    id_list = []
    topic_list = []
    for i in repo_df.index:
        topics = repo_df.loc[i,'topics']
        for t in topics:
            id_list.append(i)
            topic_list.append(t)
    df = pd.DataFrame({'id':id_list,'topic':topic_list})
    return df
            

In [95]:
info = get_repo_info(keywords, stop=20)

machine learning


Scraping top GitHub URLs...: 100%|██████████| 19/19 [00:06<00:00,  2.74it/s]


90
90
api


Scraping top GitHub URLs...: 100%|██████████| 19/19 [00:02<00:00,  7.11it/s]


0
90


In [101]:
info

,name,stargazers_count,forks_count,subscribers_count,topics,language,created_at,updated_at
21872392,awesome-machine-learning,53932,13257,3426,[],Python,2014-07-15T19:11:19Z,2022-04-12T23:20:12Z
27595858,MachineLearning,4352,3037,455,[],Python,2014-12-05T14:53:17Z,2022-04-12T21:01:55Z
50450433,machine-learning,3661,6340,433,[],Jupyter Notebook,2016-01-26T18:41:03Z,2022-04-11T15:33:16Z
86140645,Machine-Learning,6016,4434,250,"[adaboost, adaboost-algorithm, decision-tree, ...",Python,2017-03-25T07:32:37Z,2022-04-13T01:07:16Z
71154529,MachineLearning_Python,4630,2032,240,[],Python,2016-10-17T15:44:59Z,2022-04-12T17:21:38Z
...,...,...,...,...,...,...,...,...
298262517,start-machine-learning,2562,344,115,"[artificial-intelligence, cheat-sheets, course...",None,2020-09-24T11:48:46Z,2022-04-13T01:34:44Z
153575541,Hands-on-Machine-Learning,1422,441,72,[],Jupyter Notebook,2018-10-18T06:40:16Z,2022-04-09T06:20:23Z
48062442,Coursera-Machine-Learning,845,549,60,"[andrew-ng, coursera-machine-learning, predict...",Jupyter Notebook,2015-12-15T18:50:37Z,2022-04-11T04:48:29Z
24913377,Azure-MachineLearning-DataScience,395,375,194,[],HTML,2014-10-07T21:55:19Z,2022-04-10T11:10:55Z


In [100]:
topic_df = topic_relationship_table(info)
topic_df

,id,topic
0,86140645,adaboost
1,86140645,adaboost-algorithm
2,86140645,decision-tree
3,86140645,knn
4,86140645,logistic
...,...,...
529,298262517,youtube
530,298262517,youtube-playlist
531,48062442,andrew-ng
532,48062442,coursera-machine-learning


In [47]:
repo_info

{'id': 155662306,
 'node_id': 'MDEwOlJlcG9zaXRvcnkxNTU2NjIzMDY=',
 'name': 'homemade-machine-learning',
 'full_name': 'trekhleb/homemade-machine-learning',
 'private': False,
 'owner': {'login': 'trekhleb',
  'id': 3000285,
  'node_id': 'MDQ6VXNlcjMwMDAyODU=',
  'avatar_url': 'https://avatars.githubusercontent.com/u/3000285?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/trekhleb',
  'html_url': 'https://github.com/trekhleb',
  'followers_url': 'https://api.github.com/users/trekhleb/followers',
  'following_url': 'https://api.github.com/users/trekhleb/following{/other_user}',
  'gists_url': 'https://api.github.com/users/trekhleb/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/trekhleb/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/trekhleb/subscriptions',
  'organizations_url': 'https://api.github.com/users/trekhleb/orgs',
  'repos_url': 'https://api.github.com/users/trekhleb/repos',
  'events_url': 'https://api.github.c